<p style = "font-size : 42px; color : #393e46 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #00adb5; border-radius: 5px 5px;"><strong>Sign Language Identification</strong></p>

## What are Sign languages ?
<img style="margin: auto; float: center;  border:5px solid #ffb037; width:80%; height : 300px;"  src="https://image.shutterstock.com/image-photo/woman-showing-letters-asl-on-260nw-1305207925.jpg">


<br>
Sign languages are languages that use the visual-manual modality to convey meaning. Sign languages are expressed through manual articulations in combination with non-manual elements. Sign languages are full-fledged natural languages with their own grammar and lexicon .

## Problems Explaination
We are given all the different gasture images considered in ASL (Americal Sign Language) and we need to come up with a machine learning model wich can classifiy the images correctly ,or can tell us the class to which the given image belongs.
Basically we will be building a image classification model . 
<br>
## Data information
we are given 2 comma saperated file(.csv) each on them contains some rows and 785 columns
* from 2nd columns onwards each column represents the pixel values associated , representing a 28x28 grayscale image
* first column in each row represnts label with the image
* There are total 24 lables (in american sign language) A-I,K-Y means A-Z except J and Z

we will be using cnn to solve this problem .

## What is an convolution neural network
A Convolutional Neural Network (ConvNet/CNN) is a Deep Learning algorithm which can take in an input image, assign importance (learnable weights and biases) to various aspects/objects in the image and be able to differentiate one from the other.
<br>
A Cnn consists of 4 types of nueral layers (collection of 'nodes' operating together at a specific depth within a neural network) .
* **Convolutional Layer** :  Used to detect features
* **Non-Linearity Layer** :  Introducing non-linearity to the system
* **Pooling (Downsampling) Layer** :  Reduces weight count and checks fitness
* **Flattening Layer** :  Prepares data for Classical Neural Network
* **Fully-Connected Layer** :  Standard Neural Network used in classification
I'll not go in depth of each of these layers we will be more focused towards solving this problem,i.e implementation part.

# import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import accuracy_score,confusion_matrix
from keras.callbacks import ReduceLROnPlateau


In [ ]:
#reading data
train_data = pd.read_csv("../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv")
print(train_data.shape)
train_data.head()

In [ ]:
test_data = pd.read_csv("../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")
print(test_data.shape)
test_data.head()

basically in our dataset we are given lable for the corrosponding gasture and 784 (28 x 28) pixel values represting an image <br>
* Training data contains 27455 images 
* test data contains 7172 images 

# Data preprocessing
* converting array to images(tensors)

* visulaizing lables and making sure that dataset is balanced
* performing one hot encoding for lables

In [ ]:
y_train = train_data['label']
y_test = test_data['label']
del train_data['label']
del test_data['label']

In [ ]:
unique_labels = y_train.unique()
unique_labels = np.sort(unique_labels)
unique_labels

In [ ]:
# visulaizing the data
plt.figure(figsize=(15,6))
sns.set_style("darkgrid");
sns.countplot(y_train);

This really nice visual tells us that there are nearly 1k examples for each class output so we can consider this dataset as a balanec dataset because there's no class suffering from very less or too much examples 

Now the question arrises that why we didnt check the same thing for test data .... and the ans is we dont really need that <span>&#128516;</span> .<br>
In our training set we check the balance of the data coz we want to learn our model perfactly for each class we dont want it to be biased for a subset of possible classes, while in test set the whole paper can be from linear algebra , no worries in that case


In [ ]:
label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.fit_transform(y_test)

y_train[:2]

so basically `LabelBinarizer` performed OHE sort of stuff on our training data .<br>
Now let's write a function wich can take our array of images and can return a well shaped wranggled array .

In [ ]:
def preprocess_image(x):
    
    """
    we know that the pixcel values lies between 0-255 but it is obsearved that models performs exceptionally well if we scale pixel values
    between 0-1"""
    x = x/255
    x = x.reshape(-1,28,28,1) # convertin it into 28 x 28 gray scaled image
    
    return x
    

In [ ]:
train_x = preprocess_image(train_data.values)
test_x = preprocess_image(test_data.values)

let's write a function which<br>
* takes images array as input
* displays 10 images from it(first 10)

In [ ]:
def show_images(images,labels):
    fig,ax = plt.subplots(2,5)
    fig.set_size_inches(10, 6)
    k =0
    for i in range(2):
        for j in range(5):
            ax[i,j].imshow(images[k] , cmap='gray')
            ax[i,j].set_title(str(unique_labels[np.argmax(y_train[k])]))
            k = k+1;
    plt.tight_layout()

    

In [ ]:
#let's see first 10 images from training set
show_images(train_x,y_train)

In [ ]:
#let's visualize test images as well
show_images(test_x,y_test)

so far we have done a lot , we got our images ready right!

## Data Augmentation
Data augmentation is a strategy that enables practitioners to significantly increase the diversity of data available for training models, without actually collecting new data. Data augmentation techniques such as cropping, padding, and horizontal flipping are commonly used to train large neural networks.

This can expand our dataset artifically and make it robust , that couses variety in data and this can save us from overfitting.

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(train_x)

## Buidling the CNN model
we will be building a CNN model to classify images , as usual we will try a few of conv2d layers along with batchNormalization and regulization(dropout) and then fallten layer then dense layer .


In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=75, max_value=200, step=25),
        kernel_size=(3,3),
        activation='relu',
        
        input_shape=(28,28,1)
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2),strides=2,padding='same'),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=50, max_value=125, step=25),
        kernel_size=(3,3),
        activation='relu',
    ),
    
    keras.layers.Dropout(
        rate = hp.Choice('drop_1_rate', values = [0.1,0.5])
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2),strides=2,padding='same'),
    
    
    keras.layers.Conv2D(
        filters=hp.Int('conv_3_filter', min_value=25, max_value=75, step=25),
        kernel_size=(3,3),
        activation='relu',
    ),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2,2),strides=2,padding='same'),  
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=128, max_value=1024, step=32),
        activation='relu'
    ),
    keras.layers.Dropout(
        rate = hp.Choice('drop_2_rate', values = [0.1,0.3])
    ),
    keras.layers.Dense(24, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="ASLdetection1")

In [ ]:
tuner_search.search(train_x,y_train,epochs=5,validation_data = (test_x, y_test))

well,well well, what happened in above 3 cells , isn't it looking like a magic, or it's it's going over head, relax i'll try to explaing this
<br>
In the function `build_model` we build a cnn model , we spacified some layers and some of the hyperparameters are tuned using `keras-tuner` , it took each possible set of allowed parameters and trained a model and validated on test data , yep as simple as you read it.
have a look at <a href="https://keras-team.github.io/keras-tuner/documentation/hyperparameters/">keras-tuner</a>


ok now we have tired different models and it's time to extract the best model from `tuner_search` object ...let's do this 

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0] #this will give us the best tuned model

# Now let's create a callback 
A callback is a set of functions to be applied at given stages of the training procedure. You can use callbacks to get a view on internal states and statistics of the model during training.

we will reduce learning rate when a metric has stopped improving.

Models often benefit from reducing the learning rate by a factor of 2-10 once learning stagnates. This callback monitors a quantity and if no improvement is seen for a 'patience' number of epochs, the learning rate is reduced.


In [ ]:
lr_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 3, verbose=1,factor=0.5, min_lr=0.00001)

now let's train the model 

In [ ]:
history = model.fit(datagen.flow(train_x,y_train, batch_size = 128) 
                    ,epochs = 20
                    , validation_data = (test_x, y_test)
                    , callbacks = [lr_reduction])

well, we got 100% accuracy on our test data .
# Analyse the trained model

In [ ]:
model.summary()

In [ ]:
epochs = [i for i in range(20)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
fig.set_size_inches(16,9)

ax[0].plot(epochs , train_acc , 'go-' , label = 'Training Accuracy')
ax[0].plot(epochs , val_acc , 'ro-' , label = 'Testing Accuracy')
ax[0].set_title('Training & Validation Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

ax[1].plot(epochs , train_loss , 'g-o' , label = 'Training Loss')
ax[1].plot(epochs , val_loss , 'r-o' , label = 'Testing Loss')
ax[1].set_title('Testing Accuracy & Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")
plt.show()

let's now plot the model architecture

In [ ]:
tf.keras.utils.plot_model(
    model,
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)

Now let's make some predictions on test data and see the confusion metrix

In [ ]:
predictions = model.predict(test_x)

In [ ]:
def predictions_to_labels(pred):
    labels =[]
    for p in pred:
        labels.append(unique_labels[np.argmax(p)])
    return labels

In [ ]:
y_pred_labels = predictions_to_labels(predictions)
y_test_labels = predictions_to_labels(y_test)

In [ ]:
accuracy_score(y_test_labels,y_pred_labels)

In [ ]:
cm= confusion_matrix(y_test_labels,y_pred_labels)
plt.figure(figsize=(20,20))
sns.heatmap(cm,annot=True,cmap='twilight_shifted')

<p style = "font-size : 30px; color : #03506f ; font-family : 'Comic Sans MS'; "><strong>Conclusion</strong></p> 
 * we took the data from the dataframes and preprocessed the images
 * we performed data augmentation
 * we creadted a cnn model and tuned the hyperparameters with the help of `keras-tuner` library
 * we analysed our model 
<br>
<p style = "font-size : 15px; color : #035d6f ; font-family : 'Comic Sans MS'; ">At the end me made prediction of test data and as you can see we got the accuracy of <strong>100%</strong></p> 

<p style = "font-size : 20px; color : #f55c47 ; font-family : 'Comic Sans MS'; "><strong>If you like my work, please do Upvote and if you find anything which can be improved please let me know.</strong></p> 